# 知識蒸餾（Knowledge Distillation）完整教學

本教學將展示如何使用知識蒸餾技術，將大型教師模型的知識轉移到小型學生模型。

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/joshhu/knowledge-distillation/blob/main/notebooks/knowledge_distillation_tutorial.ipynb)

## 📚 什麼是知識蒸餾？

知識蒸餾是一種模型壓縮技術：

- **教師模型**：大型、高效能的模型（如 ResNet18）
- **學生模型**：小型、輕量的模型
- **軟標籤**：教師模型輸出的機率分佈
- **溫度參數**：控制軟標籤的平滑程度

## 步驟 1: 環境設定

安裝必要的套件並 clone 專案

In [ ]:
# 安裝依賴套件
!pip install -q torch torchvision tqdm matplotlib

# Clone 專案
!git clone https://github.com/joshhu/knowledge-distillation.git
%cd knowledge-distillation

print("✓ 環境設定完成！")

## 步驟 2: 匯入必要的模組

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import CosineAnnealingLR

from data import get_cifar10_dataloaders
from models import TeacherModel, StudentModel
from distillation import DistillationLoss, DistillationTrainer
from utils import set_seed, plot_training_curves, compare_models

# 檢查 GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'使用裝置: {device}')

# 設定隨機種子
set_seed(42)

## 步驟 3: 載入 CIFAR-10 資料集

In [ ]:
train_loader, test_loader = get_cifar10_dataloaders(
    data_dir='./data',
    batch_size=128,
    num_workers=2
)

print(f"資料集: CIFAR-10")
print(f"類別數量: 10")
print(f"訓練樣本: 50000")
print(f"測試樣本: 10000")

## 步驟 4: 建立教師模型和學生模型

In [ ]:
teacher_model = TeacherModel(num_classes=10).to(device)
student_model = StudentModel(num_classes=10).to(device)

print("模型建立完成！")
compare_models(teacher_model, student_model)

## 步驟 5: 訓練教師模型

注意：為了示範，這裡只訓練少量 epochs。實際應用建議訓練 20-50 epochs。

In [ ]:
def train_teacher(model, train_loader, test_loader, epochs=10):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)
    scheduler = CosineAnnealingLR(optimizer, T_max=epochs)
    
    best_acc = 0
    for epoch in range(1, epochs + 1):
        model.train()
        for inputs, targets in train_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
        
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, targets in test_loader:
                inputs, targets = inputs.to(device), targets.to(device)
                outputs = model(inputs)
                _, predicted = outputs.max(1)
                total += targets.size(0)
                correct += predicted.eq(targets).sum().item()
        
        test_acc = 100. * correct / total
        scheduler.step()
        print(f'Epoch {epoch}: 測試準確率 = {test_acc:.2f}%')
        
        if test_acc > best_acc:
            best_acc = test_acc
    
    print(f'教師模型最佳準確率: {best_acc:.2f}%')
    return model

# 訓練教師模型（設為 5 epochs 快速示範，實際建議 20+）
teacher_model = train_teacher(teacher_model, train_loader, test_loader, epochs=5)

## 步驟 6: 配置知識蒸餾參數

In [ ]:
temperature = 3.0
alpha = 0.7
num_epochs = 10  # 示範用，實際建議 50+

distillation_loss = DistillationLoss(temperature=temperature, alpha=alpha)

optimizer = optim.SGD(
    student_model.parameters(),
    lr=0.1,
    momentum=0.9,
    weight_decay=5e-4
)

scheduler = CosineAnnealingLR(optimizer, T_max=num_epochs)

trainer = DistillationTrainer(
    teacher_model=teacher_model,
    student_model=student_model,
    train_loader=train_loader,
    test_loader=test_loader,
    distillation_loss=distillation_loss,
    optimizer=optimizer,
    device=device,
    scheduler=scheduler
)

print(f"知識蒸餾配置：")
print(f"  溫度參數: {temperature}")
print(f"  Alpha: {alpha}")
print(f"  訓練輪數: {num_epochs}")

## 步驟 7: 開始知識蒸餾訓練

In [ ]:
history = trainer.train(num_epochs=num_epochs)

torch.save(student_model.state_dict(), 'student_model_distilled.pth')
print("學生模型已儲存！")

## 步驟 8: 視覺化訓練結果

In [ ]:
plot_training_curves(history)

## 步驟 9: 效能比較

In [ ]:
def evaluate_model(model, test_loader, device):
    model.eval()
    correct = 0
    total = 0
    
    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    
    return 100. * correct / total

teacher_acc = evaluate_model(teacher_model, test_loader, device)
student_acc = evaluate_model(student_model, test_loader, device)

print("="*60)
print("最終效能比較")
print("="*60)
print(f"教師模型準確率: {teacher_acc:.2f}%")
print(f"學生模型準確率: {student_acc:.2f}%")
print(f"效能保留率: {100 * student_acc / teacher_acc:.2f}%")
print("="*60)

## 🎓 總結

恭喜！你已經完成了知識蒸餾的完整流程：

1. ✅ 載入並預處理資料
2. ✅ 建立教師和學生模型
3. ✅ 訓練教師模型
4. ✅ 配置知識蒸餾
5. ✅ 訓練學生模型
6. ✅ 評估和比較效能

## 🚀 進階練習

1. 調整溫度參數（試試 1.0, 5.0, 10.0）
2. 調整 alpha 值（試試 0.5, 0.8, 0.9）
3. 增加訓練輪數到 50-100 epochs
4. 嘗試不同的模型架構

## 📚 參考資料

- [原始論文](https://arxiv.org/abs/1503.02531) - Hinton et al., 2015
- [專案 GitHub](https://github.com/joshhu/knowledge-distillation)